<a href="https://colab.research.google.com/github/alirezash97/Galaxy-Classification/blob/master/Galaxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir .kaggle
# !ls -a
import json
token = {"username":"alirezashafaei97","key":"9cb262aa0c5658ffc4eb45857c41903c"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets list
# !kaggle datasets list -s galaxy-zoo
!kaggle datasets download -d zhuangjw/galaxy-zoo-cleaned -p /content
!unzip /content/galaxy-zoo-cleaned.zip
!pip install netCDF4

- path is now set to: {/content}
 99% 2.17G/2.19G [00:32<00:00, 149MB/s]
100% 2.19G/2.19G [00:32<00:00, 73.0MB/s]
Archive:  /content/galaxy-zoo-cleaned.zip
  inflating: galaxy_id_test.csv      
  inflating: galaxy_test.nc          
  inflating: galaxy_train.nc         
  inflating: trained_simple_cnn_galaxy.h5  
     |████████████████████████████████| 4.1MB 3.3MB/s 
     |████████████████████████████████| 327kB 30.6MB/s 


In [3]:

import numpy as np
import tensorflow as tf
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from tensorflow.keras import Model
import cv2
from tqdm import tqdm
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from numpy import expand_dims
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow.keras
from keras import backend as K


Using TensorFlow backend.


In [0]:
from netCDF4 import Dataset
f = Dataset('/content/galaxy_train.nc')
images = f.variables['image_train']
labels = f.variables['label_train']
t = Dataset('/content/galaxy_test.nc')
images_test = t.variables['image_test']


In [4]:
print(images.shape)
print(images_test.shape)

(61578, 64, 64, 3)
(79975, 64, 64, 3)


In [0]:
(train_images, train_labels) = images[:60000], labels[:60000]
(test_images, test_labels) = images[60000:61500], labels[60000:61500]


In [0]:
# # special sample #####
# special_sample_image, special_sample_label = images[1500:1501], labels[1500:1501]

In [0]:
train_images = train_images/255
test_images = test_images/255

In [0]:


def identity_block(X, f, filters, stage, block):


  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  F1, F2, F3 = filters 
  X_shortcut = X

  # first component
  X = tf.keras.layers.Conv2D(filters = F1, kernel_size=(1, 1), strides=(1, 1),
            padding = 'valid', name = conv_name_base + '2a', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # second component
  X = tf.keras.layers.Conv2D(filters = F2, kernel_size=(f, f), strides=(1, 1),
            padding = 'same', name = conv_name_base + '2b',kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # Third component
  X = tf.keras.layers.Conv2D(filters = F3, kernel_size=(1, 1), strides=(1, 1),
            padding = 'valid', name = conv_name_base + '2c',kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # Final step 
  X = tf.keras.layers.Add()([X, X_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X



In [0]:
def convolutional_block(X, f, filters, stage, block, s):
  
  
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  F1, F2, F3 = filters 
  X_shortcut = X

  # first component
  X = tf.keras.layers.Conv2D(filters = F1, kernel_size=(1, 1), strides=(s, s),
            padding = 'valid', name = conv_name_base + '2a', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # second component
  X = tf.keras.layers.Conv2D(filters = F2, kernel_size=(f, f), strides=(1, 1),
            padding = 'same', name = conv_name_base + '2b', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # Third component
  X = tf.keras.layers.Conv2D(filters = F3, kernel_size=(1, 1), strides=(1, 1),
            padding = 'valid', name = conv_name_base + '2c', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  ###### shortcut path ######
  X_shortcut = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid',
                      name = conv_name_base + '1', kernel_initializer ='random_uniform')(X_shortcut)
  X_shortcut = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)


  # Final step 
  X = tf.keras.layers.Add()([X, X_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X




In [0]:
def ResNet50(input_shape, classes):
    
    # Define the input as a tensor with shape input_shape
    X_input = tf.keras.Input(input_shape)

    
    # Zero-Padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = 'random_uniform')(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = tf.keras.layers.AveragePooling2D(pool_size=(2,2), padding='same')(X)

    # Output layer
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='sigmoid', name='fc' + str(classes),kernel_initializer ='random_uniform')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:

model = ResNet50(input_shape = (64, 64, 3), classes = 37)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(train_images, train_labels, epochs = 3, batch_size = 128)

Epoch 1/3
469/469 [==============================] - 3863s 8s/step - loss: 14.6416 - accuracy: 0.5861
Epoch 2/3
469/469 [==============================] - 3869s 8s/step - loss: 14.2915 - accuracy: 0.6675
Epoch 3/3
469/469 [==============================] - 3851s 8s/step - loss: 14.2850 - accuracy: 0.6889


In [0]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


# load model
model = load_model('/content/drive/My Drive/galaxy trained model/model.h5')
# summarize model.
model.summary()


In [20]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(train_images)


history = model.fit_generator(datagen.flow(train_images, train_labels, batch_size=256),
                    steps_per_epoch=len(train_images) / 256, epochs=1)

235/234 [==============================] - 4087s 17s/step - loss: 14.3031 - accuracy: 0.7061


In [21]:
scores = model.evaluate(test_images, test_labels)
model.save("/content/drive/My Drive/galaxy trained model/Augmented_model.h5")
print("Saved model to disk")

47/47 [==============================] - 10s 214ms/step - loss: 17.9509 - accuracy: 0.5907
Saved model to disk


In [0]:
prediction = model.predict(test_images, batch_size=64)

In [0]:
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


special_prediction = model.predict(special_sample_image)
# Create Decision Tree classifer object
clf = DecisionTreeRegressor()

# Train Decision Tree Classifer
clf = clf.fit(prediction, test_labels)

#Predict the response for test dataset
y_pred = clf.predict(special_prediction)





In [0]:

print(y_pred)
print(special_sample_label)

[[0.         1.         0.         0.059619   0.94038099 0.78539115
  0.15498984 0.92826325 0.01211775 0.         0.42122674 0.45301819
  0.06613605 0.185486   0.81451398 0.         0.         0.
  0.1112916  0.         0.0370972  0.0370972  0.         0.
  0.         0.059619   0.         0.         0.27397969 0.62240887
  0.03187563 0.         0.85712206 0.04611148 0.         0.
  0.02502969]]
[[0.855726   0.098069   0.046205   0.         0.098069   0.
  0.098069   0.         0.098069   0.         0.03400798 0.03092018
  0.03314085 0.419136   0.580864   0.6955675  0.16015853 0.
  0.         0.02978842 0.         0.         0.2395664  0.14978118
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]]


In [0]:
a = y_pred - special_sample_label
b = special_prediction - special_sample_label
print(np.sum(np.square(a)))
print(np.sum(np.square(b)))

5.998199333426978
5.2012835


In [0]:
scores1 = model.evaluate(special_sample_image, special_sample_label)

1/1 [==============================] - 0s 1ms/step - loss: 11.7066 - accuracy: 0.0000e+00
